# Introduction to C++ for Python Programmers

We look at primary constructs that are common to nearly all programming languages:

- Using code libraries
- The main function
- Standard i/o
- Data types and operators
- Conditionals
- Loops

Next, we look at additional useful features of C++ including:

- Functions and their parameters
- File operations
- Exception handling

## Why learn C++?

These other more formal languages have some advantages of their own. First, is speed: For very large programs C and C++ are likely to give you the best performance. Second, is their maintainability. Python requires you to remember certain things. For example if you set variable t to reference a turtle, and forget later that t is a turtle but try to invoke a string method on it, you will get an error. C++ protects you from this kind of error by forcing you to be upfront and formal about the type of object each variable is going to refer to.

C++ is an industrial strength programming language and is very often used today for large systems by large groups of people. C++ is particularly good at interacting directly with computer hardware, making execution very fast. C++ supports the four primary features of OOP: abstraction, inheritance, polymorphism, and encapsulation. C++ allows the programmer to create and use a data type called a **pointer** explicitly, which can increase control over both memory and performance under certain circumstances. Because C++ is fast, it is currently the language of choice for virtual reality. Also, because C++ is fast, it is the language of choice of many 2D and 3D game engines. For all of the above reasons, even though C++ is an older language, it is still one of the top listed in job advertisements.

## Hello, world!

The time honored tradition. First, we recall in Python:

In [1]:
print("Hello, world!")

Hello, world!


This can be made into a more complicated version:

In [2]:
def main():
    print("Hello, world!")

main()

Hello, world!


The version in C++:

In [3]:
%%writefile ./code/01_01.cpp
#include <iostream>

int main(){
    std::cout << "Hello, world!\n";
    return 0;
}

Overwriting ./code/01_01.cpp


In [4]:
!g++-15 ./code/01_01.cpp -o ./code/01_01
!./code/01_01

Hello, world!


**NOTE:** Compiling and then running. In my case, I have `gcc` (v15) installed using Homebrew on a MacBook. That's a lot of writing, so we define a helper function for running the code in a jupyter cell:

In [5]:
from IPython.display import display, HTML
from IPython import get_ipython
from pathlib import Path

def get_latest_file(folder: str):
    p = Path(folder)
    fn = max(p.glob("**/*.cpp"), key=lambda f: f.stat().st_mtime)
    return fn.name.replace(".cpp", "")

def run(fn: str=None, comp: bool=True):
    fn = get_latest_file("./code") if fn is None else fn
    cmd = f"!./code/{fn}"
    if comp:
        cmd = f"!g++-15 ./code/{fn}.cpp -o ./code/{fn}\n" + cmd

    # run
    print((cmd + "\n").replace("!", "$ "))
    get_ipython().run_cell(cmd)


run();  # gets last modified .cpp source file when fn unspecified

$ g++-15 ./code/01_01.cpp -o ./code/01_01
$ ./code/01_01

Hello, world!


The ff allows us to skip writing `std::` every time we use a component in the C++ standard library:

In [6]:
%%writefile ./code/01_02.cpp
#include <iostream>
using namespace std;

int main(){
    cout << "Hello World!\n";
    return 0;
}

Overwriting ./code/01_02.cpp


In [7]:
run();

$ g++-15 ./code/01_02.cpp -o ./code/01_02
$ ./code/01_02

Hello World!


## Compilation

To run the C++ programs in the previous section, we compiled it using `g++`. This allows (1) early error detection (e.g. syntax or type errors), and (2) faster program execution. The job of the compiler is to turn your C++ code into language that your machine can understand. We call the code that the computer understands **machine code**. The computer interprets the machine code much like the Python interpreter interprets your Python. However, since machine code is much closer to the native language of the computer, it can run faster.

**Sidenote.** Python code is not directly converted into machine code that a computer's CPU can execute. Instead, Python code is first compiled into an intermediate form called **bytecode**, which is then interpreted by the [Python Virtual Machine](https://blog.codingconfessions.com/p/cpython-vm-internals) (PVM). The PVM translates the bytecode into machine code instructions that the specific CPU can understand and run.

### Headers and libraries

**Preprocessor directives** in C++ appear as statements preceded by the hash sign `#`. These tell the preprocessor which file, header, or library to make available to the compiler. For example, `#include <iostream>` will make sure that the iostream library is available at compile time. Here, the term **header** is used for a type of C++ file that contains definitions of functions and variables, but *not* the function implementations.

This is similar to Python imports. But note that there are two ways to use `#include` in C++:

```c++
#include <libraryname>
#include "filename"
```

Here the angle-brackets `<>` are used to include libraries or headers provided by the implementation, such as the headers in the standard library (iostream, string, etc.). The double quotes `"` are used for headers and files not provided by the implementation (i.e. compiler and your version of the standard library).

```c++
#include <vector>      // provided by the C++ standard library
#include "utils.h"     // a custom file in your project
```



### Linking

The `<iostream>` header provides declarations for input / output classes like `std::cin`, `std::cout`, and `std::endl`. These are defined in the standard C++ library, which is automatically linked by the compiler. You don't need to manually link any `.a` or `.so` file — the compiler links the C++ standard library by default. Some libraries require explicit linking:

```bash
g++ main.cpp -lm     # link the math library (libm)
g++ main.cpp         # links the standard C++ library automatically
```

In Python when you do `import math`,
you're not recompiling the `math` module. Python just links your code to an already compiled `.so` or `.pyd` file. The same idea applies in C++ — but it happens at compile + link time instead of import time. Pre-compiled libraries (compiled by the compiler vendor or maintainer) allow compilation of new code to be faster (i.e. compile once, reuse multiple times). Going back to `<iostream>`, this header declares `std::cout`.
The implementation of `std::cout` lives in a compiled library like `libc++.dylib` (macOS). The compiler knows where to find and link that binary automatically.


In [8]:
import time

t0 = time.time()
run("01_01")
print(time.time() - t0)

$ g++-15 ./code/01_01.cpp -o ./code/01_01
$ ./code/01_01

Hello, world!
0.5863211154937744


Not compiling results in ~8-9x speedup (on my machine):

In [9]:
t0 = time.time()
run("01_01", comp=False)
print(time.time() - t0)

$ ./code/01_01

Hello, world!
0.11997699737548828


## The `main` function

Unlike Python, every C++ program must have a `main` function which begins with `int main()`. This `main` function is called implicitly instead of explicitly like we must do in Python when we have a main function. This is why you do not see an explicit function call invoking main.

This is very similar to the Python version. Except that we have to specify the type of the return value. And that the integer return value must be specified to some integer value (customary to return `0` whenever there is no error, and specific return values for error handling). I tried to do `void main()` in C++ which is closer in spirit, but it results in an error: 

```
error: '::main' must return 'int'
```

A more accurate Python version should then be:

In [10]:
def main():
    print("Hello, world!")
    return 42

main()

Hello, world!


42

That translates to:

In [11]:
%%writefile "./code/01_03.cpp"
#include <iostream>
using namespace std;

int main(){
    cout << "Hello World!\n";
    return 42;
}

Overwriting ./code/01_03.cpp


In [12]:
!g++-15 ./code/01_03.cpp -o ./code/01_03
!./code/01_03; echo $?

Hello World!
42


## Standard I/O

**Standard output.** Let's take a step back and look at `cout`. This actually stands for "character output" which sends character data to the screen. The operator `<<` directs the string to this output device. Note that multiple uses of `<<` onto `cout` corresponds to concatenating the strings:

In [13]:
%%writefile "./code/01_04.cpp"
#include <iostream>
using namespace std;

int main(){
    cout << "Ever heard of rubber duck debugging?" << endl;
    cout << "                __     " << endl;
    cout << "              <(o )___-" << endl;
    cout << "               ( .__> /" << endl;
    cout << "                `----' " << endl;    
    return 0;
}

Overwriting ./code/01_04.cpp


**Q.** Why use `endl` when we can just `+ "\n"`? It turns out that these have different side-effects. Inserting a newline character does not **flush** the output buffer. On the other hand, using `std::endl` inserts a newline and flushes the output stream (i.e., forces it to write to the terminal immediately). Flushing can be useful in interactive programs (e.g., before `cin >> ...`), but adds overhead.


In [14]:
run()

$ g++-15 ./code/01_04.cpp -o ./code/01_04
$ ./code/01_04

Ever heard of rubber duck debugging?
                __     
              <(o )___-
               ( .__> /
                `----' 


**Standard input.** Like `cout`, `cin` is also a character stream with `>>` used to direct input onto it. For standard input, we can pipe the input via terminal: 

In [15]:
%%writefile "./code/01_05.cpp"
#include <iostream>
using namespace std;

int main(){
    string resp;
    cout << "\nEver heard of rubber duck debugging?" << endl;
    cin  >> resp;
    if ((resp[0] == 'Y') || (resp[0] == 'y')){
        cout << "cool B)" << endl;
    }
    else {
        cout << "Here you go." << endl;
        cout << "                __     " << endl;
        cout << "              <(o )___-" << endl;
        cout << "               ( .__> /" << endl;
        cout << "                `----' " << endl;    
    }
        
    return 0;
}

Overwriting ./code/01_05.cpp


**Remark.** Using `"Y"` fails. This is because `response[0]` is a char that should be compared with another char `Y`. Meanwhile, `"Y"` is a `const char*` (C-style string).

In [16]:
!g++-15 ./code/01_05.cpp -o ./code/01_05
!echo "yes" | ./code/01_05
!echo "no" | ./code/01_05


Ever heard of rubber duck debugging?
cool B)

Ever heard of rubber duck debugging?
Here you go.
                __     
              <(o )___-
               ( .__> /
                `----' 


## Type declarations

In the above examples, we declare new variables with a type, e.g. `string resp;`. Just like functions, all variables in C++ must be declared before use, and they cannot change type. This is known as **static typing**. 

The line `string resp` essentially tells the compiler to set aside sufficient space for a floating point number, and to name this memory location `resp`. Then whatever the user types in will be stored in the `resp` variable.

## Conclusion

We can now look back at the initial program with better understanding of its elements:

```c++
/* 
This hello world program demonstrates the C++ concepts
of commenting, using libraries, and using output.
*/

#include <iostream>               // ①
using namespace std;              // ② 

int main(){                       // ③   
    cout << "Hello World!\n";     // ④ 
    return 0;                     // ⑤
}
```

1. directive for including iostream header
2. makes `std::` implicit
1. `main()` must exist & return an int
2. character stream, print to std output device
3. `0` indicates program ended correctly.